# Neural Network From Scratch

In [112]:
import pandas as pd
import numpy as np
import random

random.seed(5)

# Initialise Network Parameters

In [113]:
def initialise_params(dims):
    """
    ## Generates units in each layer of the network ##
    Inputs: number of units in each layer dims = [n0, n1, n2, ..., nL]
    output: W = {"W1": W1, ..., "WL": WL}, b = {"b1": b1, ..., "bL": bL}
    """
    # Number of layers
    L = len(dims) - 1
    
    # Dictionary of parameters
    W = {}
    b = {}
    for l in range(L):
        W["W"+str(l+1)] = np.random.randn(dims[l+1], dims[l])
        b["b"+str(l+1)] = np.zeros((dims[l+1], 1))
    
    return W, b

In [114]:
# Test initialise_params()
dims = [2, 4, 4, 2]
W, b = initialise_params(dims)

print("------------------------------")
print("## Test initialise_params() ##")
print("------------------------------\n\n")

for i in range(1,len(dims)):
    print("Layer {}:".format(i))
    print("----------\n")
    print("W"+str(i)+" = {}".format(W["W"+str(i)]))
    print("b"+str(i)+" = {}\n".format(b["b"+str(i)]))

------------------------------
## Test initialise_params() ##
------------------------------


Layer 1:
----------

W1 = [[-0.3035558  -0.27386325]
 [ 0.02504767 -2.42048771]
 [-1.06034014  0.90046174]
 [ 0.84970209 -0.70458156]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Layer 2:
----------

W2 = [[ 0.58560923 -1.06727298 -0.33213454  1.31991766]
 [ 0.88380192  0.04855433  0.08540305  0.11245358]
 [-0.87605494  1.71359624 -1.17368316  0.38334488]
 [ 0.62631414 -0.12176295 -1.68946266  0.62579032]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Layer 3:
----------

W3 = [[-2.87899745 -2.09551492 -0.24829506 -0.15788123]
 [ 0.35145937  1.5796131  -0.78909547 -0.87589481]]
b3 = [[0.]
 [0.]]



# Forward Linear Calculation

In [115]:
def forward_linear(A_prev, W, b):
    """
    ## Calculate linear equation for forward pass ##
    Inputs: A = previous layer activation, W = current weights, b = current biases
    Output: Z = W•A + b
    """
    
    # Linear Equation
    Z = np.dot(W, A_prev) + b
    
    return Z

In [116]:
# Test forward_linear()
x = np.array([[2], [5]])
W1 = units["W1"]
b1 = units["b1"]

Z1 = forward_linear(x, W1, b1)

print("------------------------------")
print("## Test forward_linear() ##")
print("------------------------------\n")

print("\nInputs: \n----------")
print("x = ", x)
print("\nW1 = ", W1)
print("\nb1 = ", b1)

print("\nOutput: \n----------")
print("Z1 = ", Z1)

------------------------------
## Test forward_linear() ##
------------------------------


Inputs: 
----------
x =  [[2]
 [5]]

W1 =  [[-0.72148419 -0.06346357]
 [ 0.43440561  0.58890866]
 [ 0.07431876  0.23153176]
 [-0.02325687  1.16768063]]

b1 =  [[0.]
 [0.]
 [0.]
 [0.]]

Output: 
----------
Z1 =  [[-1.76028624]
 [ 3.81335449]
 [ 1.30629634]
 [ 5.79188939]]


# Forward Activation Calculation

In [117]:
########################
# Activation Functions #
########################
    

# Sigmodal Activation
def Sigmoid(Z):
    return 1/(1 + np.exp(-Z))

# Swish Activation
def Swish(Z):
    return Z/(1 + np.exp(-Z))

# Hyperbolic Tangent
def Tanh(Z):
    return np.tanh(Z)

# Rectified Linear Unit
def ReLU(Z):
    return np.maximum(0, Z)

# Leaky Rectified Linear Unit
def L_ReLU(Z):
    return np.maximum(0.01*Z, Z)

# # Parametric Rectified Linear Unit
# def P_ReLU(Z):
#     return np.maximum(0.05*Z, Z)


############################
# Forward pass calculation #
############################

def forward_activation(Z, g):
    """ 
    ## Applies specified activation function to Z ##
    Inputs: Z = linear forward pass, g = activation function
    Output: A = g(z) 
    """

    # Dictionary of activations functions
    activations = {
        'sigmoid': Sigmoid,
        'swish'  : Swish,
        'tanh'   : Tanh,
        'relu'   : ReLU,
        'l_relu' : L_ReLU
    }

    # Call activation on Z
    try:
        A = activations[g](Z)
        return A
    except:
        display(print("\n----------------\nInvalid function\n----------------\n"))
        display(print(g))
        display(print("----------------\nValid activations:\n----------------"))
        for key in activations.keys():
            display(print(key))
        pass

In [118]:
# Test forward_activation()
Z1 = Z1
g = "relu"

print("------------------------------")
print("## Test forward_activation() ##")
print("------------------------------\n")

print("\nInputs: \n----------")
print("Z1 = ", Z1)
print("\ng = ", g)

print("\nOutput: \n----------")
A1 = forward_activation(Z1, g)
print("A = ", A1)

------------------------------
## Test forward_activation() ##
------------------------------


Inputs: 
----------
Z1 =  [[-1.76028624]
 [ 3.81335449]
 [ 1.30629634]
 [ 5.79188939]]

g =  relu

Output: 
----------
A =  [[0.        ]
 [3.81335449]
 [1.30629634]
 [5.79188939]]


# Model Forward Propagation

In [119]:
def foward_propagation(X, W, b, g):
    """
    ## Completes a full forward pass through Network ##
    Inputs: X = Full Feature Set, W = Weights in Network, 
            b = Biases in Network, g = All Activation Functions
    Output: A, Z
    """
    assert(len(W) == len(b))
    assert(len(b) == len(g))
    assert(len(g) == len(W))
    
    L = len(W)
    A_prev = X
    A = {}
    Z = {}
    
    for l in range(L):
        # Parameters
        bl = b["b"+str(l+1)]
        Wl = W["W"+str(l+1)]
        
        # Current activation 
        gl = g["g"+str(l+1)]
        
        # Forward Linear Pass
        Zl = forward_linear(A_prev, Wl, bl)
        Al = forward_activation(Zl, gl)
        
        # Chase Values
        A["A"+str(l+1)] = Al
        Z["Z"+str(l+1)] = Zl
        
        A_prev = Al   
    
    return A, Z

In [121]:
# Test forward_propagation()

## Specify inputs
X = x
dims = [X.shape[0], 4, 4, 2, 1]
g = {
    "g1": 'relu',
    "g2": 'relu',
    "g3": 'relu',
    "g4": 'sigmoid'
}

## Run initialise_params()
W, b = initialise_params(dims)

print("------------------------------")
print("## Test forward_activation() ##")
print("------------------------------\n\n")

print("N = ", dims)
print("-------------------\n-------------------\n\n")

## Run foward_propagation()
A, Z = foward_propagation(X, W, b, g)

for i in range(1,len(dims)):
    print("Layer {}:".format(i))
    print("----------\n")
    print("W"+str(i)+" = {}".format(W["W"+str(i)]))
    print("b"+str(i)+" = {}\n".format(b["b"+str(i)]))
    print("Z"+str(i)+" = {}".format(Z["Z"+str(i)]))
    print("A"+str(i)+" = {}\n\n".format(A["A"+str(i)]))

------------------------------
## Test forward_activation() ##
------------------------------


N =  [2, 4, 4, 2, 1]
-------------------
-------------------


Layer 1:
----------

W1 = [[-1.4457343   1.03734989]
 [ 0.76852187 -1.53801337]
 [ 0.81016917 -0.78179952]
 [ 1.26777548  0.24812484]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]

Z1 = [[ 2.29528083]
 [-6.15302312]
 [-2.28865924]
 [ 3.77617513]]
A1 = [[2.29528083]
 [0.        ]
 [0.        ]
 [3.77617513]]


Layer 2:
----------

W2 = [[ 1.01882147  0.86149989  0.39889563 -0.10066499]
 [-1.44505897 -0.51109455 -1.50177042  0.08028097]
 [ 0.67315815 -1.22538675 -0.9784412  -0.28615619]
 [ 0.42723869  0.43352469 -0.66567287 -0.6446957 ]]
b2 = [[0.]
 [0.]
 [0.]
 [0.]]

Z2 = [[ 1.95835274]
 [-3.01366116]
 [ 0.46451112]
 [-1.45385112]]
A2 = [[1.95835274]
 [0.        ]
 [0.46451112]
 [0.        ]]


Layer 3:
----------

W3 = [[-1.6408348  -1.66480963 -1.21713812 -1.37673776]
 [-1.42444564 -0.08732428  0.53161913  0.28062724]]
b3 = [[0.]
 [0.]]

Z3 = 

# Calculate Cost

Model Backward Propagation

Backward Linear Activation

Backwards Linear Calculation

Update Network Parameter

Put it all together